In [1]:
from nowcasting_dataset.data_sources import NWPDataSource
from pathlib import Path
import pandas as pd

In [2]:
BUCKET = Path('solar-pv-nowcasting-data')
NWP_BASE_PATH = BUCKET / 'NWP/UK_Met_Office/UKV_zarr'

In [3]:
nwp_ds = NWPDataSource(
    filename=f'gs://{NWP_BASE_PATH}',
    history_len=0,
    forecast_len=1,
    image_size_pixels=2,
    meters_per_pixel=2_000)

In [4]:
%%time
nwp_ds.open()

CPU times: user 6.6 s, sys: 323 ms, total: 6.92 s
Wall time: 11.5 s


In [ ]:
%%time
datetimes = nwp_ds.datetime_index()

In [7]:
nwp_ds.data

,Array,Chunk
Bytes,2.83 TiB,58.87 MiB
Shape,"(10, 5442, 37, 704, 548)","(1, 2, 20, 704, 548)"
Count,191960 Tasks,54460 Chunks
Type,float32,numpy.ndarray


In [6]:
%%time
example = nwp_ds.get_example(
    x_meters_center=100000,
    y_meters_center=100000,
    t0_dt=pd.Timestamp('2019-11-16 10:05:00')
)


CPU times: user 2.59 s, sys: 530 ms, total: 3.12 s
Wall time: 1.18 s


In [ ]:
example['nwp'].shape

In [5]:
%%time
mean = nwp_ds.data.isel(init_time=slice(0, 10)).mean(dim=['step', 'x', 'init_time', 'y']).compute()
mean

CPU times: user 30.7 s, sys: 4.4 s, total: 35.1 s
Wall time: 6.94 s


<xarray.DataArray 'stack-876e897afd763ccc422128d7c9c72eea' (variable: 10)>
array([2.8041010e+02, 1.6854691e+01, 6.7529683e-05, 8.1832832e+01,
       7.1233767e-03, 8.8566933e+00, 4.3474598e+04, 4.9820110e+01,
       4.8095409e+01, 4.2833260e+01], dtype=float32)
Coordinates:
  * variable  (variable) <U5 't' 'dswrf' 'prate' 'r' ... 'vis' 'lcc' 'mcc' 'hcc'

In [6]:
std = nwp_ds.data.isel(init_time=slice(0, 10)).std(dim=['step', 'x', 'init_time', 'y']).compute()
std

<xarray.DataArray 'stack-876e897afd763ccc422128d7c9c72eea' (variable: 10)>
array([2.5812180e+00, 4.1278820e+01, 2.7507244e-04, 9.0967312e+00,
       1.4110464e-01, 4.3616886e+00, 2.3853148e+04, 3.8900299e+01,
       4.2830105e+01, 4.2778091e+01], dtype=float32)
Coordinates:
  * variable  (variable) <U5 't' 'dswrf' 'prate' 'r' ... 'vis' 'lcc' 'mcc' 'hcc'